In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob

from sklearn.model_selection import KFold
import lightgbm as lgb

import psutil
import random
import os
import time
import sys
import math
from contextlib import contextmanager

In [2]:
N_SPLITS = 20
SEED = 1234

In [3]:
LOG_PATH = Path("./log/")
LOG_PATH.mkdir(parents=True, exist_ok=True)

In [4]:
@contextmanager
def timer(name: str):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    try:
        yield
    finally:
        m1 = p.memory_info()[0] / 2. ** 30
        delta = m1 - m0
        sign = '+' if delta >= 0 else '-'
        delta = math.fabs(delta)
        print(f"[{m1:.1f}GB({sign}{delta:.1f}GB): {time.time() - t0:.3f}sec] {name}", file=sys.stderr)


def set_seed(seed=1234):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

    
def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]


In [18]:
# !pwd
set_seed(SEED)
feature_dir = "/Users/revanth/Downloads/IndoorLocationNavigation/wifi_features"
train_files = sorted(glob.glob(os.path.join(feature_dir + '/train/', '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir + '/test/', '*_test.csv')))
subm = pd.read_csv('./sample_submission.csv', index_col=0)

In [21]:
lgb_params = {'objective': 'root_mean_squared_error',
              'boosting_type': 'gbdt',
              'n_estimators': 50000,
              'learning_rate': 0.1,
              'num_leaves': 90,
              'colsample_bytree': 0.4,
              'subsample': 0.6,
              'subsample_freq': 2,
              'bagging_seed': SEED,
              'reg_alpha': 8,
              'reg_lambda': 2,
              'random_state': SEED,
              'n_jobs': -1
              }

lgb_f_params = {'objective': 'multiclass',
                'boosting_type': 'gbdt',
                'n_estimators': 50000,
                'learning_rate': 0.1,
                'num_leaves': 90,
                'colsample_bytree': 0.4,
                'subsample': 0.6,
                'subsample_freq': 2,
                'bagging_seed': SEED,
                'reg_alpha': 10,
                'reg_lambda': 2,
                'random_state': SEED,
                'n_jobs': -1
                }

Splitting the data into train and validation and passed them through LGBm and then saving the predicitions into pred

In [22]:
score_df = pd.DataFrame()
oof = list()
predictions = list()
for n_files, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0)
    test_data = pd.read_csv(test_files[n_files], index_col=0)

    oof_x, oof_y, oof_f = np.zeros(data.shape[0]), np.zeros(data.shape[0]), np.zeros(data.shape[0])
    preds_x, preds_y = 0, 0
    preds_f_arr = np.zeros((test_data.shape[0], N_SPLITS))

    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(data.iloc[:, :-4])):
        X_train = data.iloc[trn_idx, :-4]
        y_trainx = data.iloc[trn_idx, -4]
        y_trainy = data.iloc[trn_idx, -3]
        y_trainf = data.iloc[trn_idx, -2]

        X_valid = data.iloc[val_idx, :-4]
        y_validx = data.iloc[val_idx, -4]
        y_validy = data.iloc[val_idx, -3]
        y_validf = data.iloc[val_idx, -2]

        modelx = lgb.LGBMRegressor(**lgb_params)
        with timer("fit X"):
            modelx.fit(X_train, y_trainx,
                       eval_set=[(X_valid, y_validx)],
                       eval_metric='rmse',
                       verbose=True,
                       early_stopping_rounds=5
                       )

        modely = lgb.LGBMRegressor(**lgb_params)
        with timer("fit Y"):
            modely.fit(X_train, y_trainy,
                       eval_set=[(X_valid, y_validy)],
                       eval_metric='rmse',
                       verbose=True,
                       early_stopping_rounds=5
                       )
        modelf = lgb.LGBMClassifier(**lgb_f_params)
        with timer("fit F"):
            modelf.fit(X_train, y_trainf,
                       eval_set=[(X_valid, y_validf)],
                       eval_metric='multi_logloss',
                       verbose=True,
                       early_stopping_rounds=5
                       )

        oof_x[val_idx] = modelx.predict(X_valid)
        oof_y[val_idx] = modely.predict(X_valid)
        oof_f[val_idx] = modelf.predict(X_valid).astype(int)

        preds_x += modelx.predict(test_data.iloc[:, :-1]) / N_SPLITS
        preds_y += modely.predict(test_data.iloc[:, :-1]) / N_SPLITS
        preds_f_arr[:, fold] = modelf.predict(test_data.iloc[:, :-1]).astype(int)

        score = comp_metric(oof_x[val_idx], oof_y[val_idx], oof_f[val_idx],
                            y_validx.to_numpy(), y_validy.to_numpy(), y_validf.to_numpy())
        print(f"fold {fold}: mean position error {score}")

    print("*+"*40)
    print(f"file #{n_files}, shape={data.shape}, name={os.path.basename(file)}")
    score = comp_metric(oof_x, oof_y, oof_f,
                        data.iloc[:, -4].to_numpy(), data.iloc[:, -3].to_numpy(), data.iloc[:, -2].to_numpy())
    oof.append(score)
    print(f"mean position error {score}")
    print("*+"*40)

    preds_f_mode = stats.mode(preds_f_arr, axis=1)
    preds_f = preds_f_mode[0].astype(int).reshape(-1)
    test_preds = pd.DataFrame(np.stack((preds_f, preds_x, preds_y))).T
    test_preds.columns = subm.columns
    test_preds.index = test_data["site_path_timestamp"]
    test_preds["floor"] = test_preds["floor"].astype(int)
    predictions.append(test_preds)

[1]	valid_0's rmse: 24.8357
Training until validation scores don't improve for 5 rounds
[2]	valid_0's rmse: 22.6239
[3]	valid_0's rmse: 20.6252
[4]	valid_0's rmse: 18.8593
[5]	valid_0's rmse: 17.2608
[6]	valid_0's rmse: 15.8027
[7]	valid_0's rmse: 14.5159
[8]	valid_0's rmse: 13.3662
[9]	valid_0's rmse: 12.3348
[10]	valid_0's rmse: 11.3713
[11]	valid_0's rmse: 10.5779
[12]	valid_0's rmse: 9.85932
[13]	valid_0's rmse: 9.23457
[14]	valid_0's rmse: 8.66203
[15]	valid_0's rmse: 8.13885
[16]	valid_0's rmse: 7.69973
[17]	valid_0's rmse: 7.28927
[18]	valid_0's rmse: 6.92072
[19]	valid_0's rmse: 6.63188
[20]	valid_0's rmse: 6.37428
[21]	valid_0's rmse: 6.12331
[22]	valid_0's rmse: 5.89822
[23]	valid_0's rmse: 5.7283
[24]	valid_0's rmse: 5.54908
[25]	valid_0's rmse: 5.38718
[26]	valid_0's rmse: 5.25273
[27]	valid_0's rmse: 5.14184
[28]	valid_0's rmse: 5.03925
[29]	valid_0's rmse: 4.93604
[30]	valid_0's rmse: 4.83391
[31]	valid_0's rmse: 4.74513
[32]	valid_0's rmse: 4.66128
[33]	valid_0's rmse: 4

[10.2GB(+0.0GB): 5.012sec] fit X


[1]	valid_0's rmse: 48.3368
Training until validation scores don't improve for 5 rounds
[2]	valid_0's rmse: 43.8814
[3]	valid_0's rmse: 39.9108
[4]	valid_0's rmse: 36.3489
[5]	valid_0's rmse: 33.1265
[6]	valid_0's rmse: 30.1917
[7]	valid_0's rmse: 27.5251
[8]	valid_0's rmse: 25.1017
[9]	valid_0's rmse: 23.0328
[10]	valid_0's rmse: 21.0779
[11]	valid_0's rmse: 19.3707
[12]	valid_0's rmse: 17.7853
[13]	valid_0's rmse: 16.4353
[14]	valid_0's rmse: 15.1746
[15]	valid_0's rmse: 14.0275
[16]	valid_0's rmse: 13.0473
[17]	valid_0's rmse: 12.1563
[18]	valid_0's rmse: 11.3688
[19]	valid_0's rmse: 10.6656
[20]	valid_0's rmse: 10.0379
[21]	valid_0's rmse: 9.4841
[22]	valid_0's rmse: 8.97685
[23]	valid_0's rmse: 8.53044
[24]	valid_0's rmse: 8.15693
[25]	valid_0's rmse: 7.77905
[26]	valid_0's rmse: 7.45773
[27]	valid_0's rmse: 7.17213
[28]	valid_0's rmse: 6.91701
[29]	valid_0's rmse: 6.68118
[30]	valid_0's rmse: 6.48092
[31]	valid_0's rmse: 6.31183
[32]	valid_0's rmse: 6.15089
[33]	valid_0's rmse: 6

[10.2GB(+0.0GB): 5.340sec] fit Y


[1]	valid_0's multi_logloss: 1.51386
Training until validation scores don't improve for 5 rounds
[2]	valid_0's multi_logloss: 1.2304
[3]	valid_0's multi_logloss: 1.02957
[4]	valid_0's multi_logloss: 0.870551
[5]	valid_0's multi_logloss: 0.743184
[6]	valid_0's multi_logloss: 0.641323
[7]	valid_0's multi_logloss: 0.559215
[8]	valid_0's multi_logloss: 0.487327
[9]	valid_0's multi_logloss: 0.428077
[10]	valid_0's multi_logloss: 0.377004
[11]	valid_0's multi_logloss: 0.333571
[12]	valid_0's multi_logloss: 0.295801
[13]	valid_0's multi_logloss: 0.262788
[14]	valid_0's multi_logloss: 0.234669
[15]	valid_0's multi_logloss: 0.210466
[16]	valid_0's multi_logloss: 0.189915
[17]	valid_0's multi_logloss: 0.171574
[18]	valid_0's multi_logloss: 0.155227
[19]	valid_0's multi_logloss: 0.141706
[20]	valid_0's multi_logloss: 0.129056
[21]	valid_0's multi_logloss: 0.118064
[22]	valid_0's multi_logloss: 0.108437
[23]	valid_0's multi_logloss: 0.0998632
[24]	valid_0's multi_logloss: 0.0925124
[25]	valid_0's 

...

[1]	valid_0's rmse: 32.5628
Training until validation scores don't improve for 5 rounds
[2]	valid_0's rmse: 30.061
[3]	valid_0's rmse: 27.8371
[4]	valid_0's rmse: 25.9252
[5]	valid_0's rmse: 24.0842
[6]	valid_0's rmse: 22.5224
[7]	valid_0's rmse: 21.0808
[8]	valid_0's rmse: 19.8191
[9]	valid_0's rmse: 18.7308
[10]	valid_0's rmse: 17.7188
[11]	valid_0's rmse: 16.9068
[12]	valid_0's rmse: 16.1359
[13]	valid_0's rmse: 15.424
[14]	valid_0's rmse: 14.7936
[15]	valid_0's rmse: 14.2051
[16]	valid_0's rmse: 13.7249
[17]	valid_0's rmse: 13.2794
[18]	valid_0's rmse: 12.9615
[19]	valid_0's rmse: 12.6188
[20]	valid_0's rmse: 12.3252
[21]	valid_0's rmse: 12.0045
[22]	valid_0's rmse: 11.7459
[23]	valid_0's rmse: 11.5041
[24]	valid_0's rmse: 11.3474
[25]	valid_0's rmse: 11.1941
[26]	valid_0's rmse: 11.0372
[27]	valid_0's rmse: 10.8767
[28]	valid_0's rmse: 10.7322
[29]	valid_0's rmse: 10.6131
[30]	valid_0's rmse: 10.4946
[31]	valid_0's rmse: 10.418
[32]	valid_0's rmse: 10.3074
[33]	valid_0's rmse: 10.

[10.4GB(+0.0GB): 4.352sec] fit Y


[1]	valid_0's multi_logloss: 1.64674
Training until validation scores don't improve for 5 rounds
[2]	valid_0's multi_logloss: 1.38938
[3]	valid_0's multi_logloss: 1.20572
[4]	valid_0's multi_logloss: 1.07291
[5]	valid_0's multi_logloss: 0.957508
[6]	valid_0's multi_logloss: 0.861322
[7]	valid_0's multi_logloss: 0.777907
[8]	valid_0's multi_logloss: 0.709682
[9]	valid_0's multi_logloss: 0.645548
[10]	valid_0's multi_logloss: 0.597444
[11]	valid_0's multi_logloss: 0.555716
[12]	valid_0's multi_logloss: 0.513229
[13]	valid_0's multi_logloss: 0.476238
[14]	valid_0's multi_logloss: 0.444206
[15]	valid_0's multi_logloss: 0.419517
[16]	valid_0's multi_logloss: 0.39517
[17]	valid_0's multi_logloss: 0.374292
[18]	valid_0's multi_logloss: 0.355912
[19]	valid_0's multi_logloss: 0.339659
[20]	valid_0's multi_logloss: 0.323939
[21]	valid_0's multi_logloss: 0.308327
[22]	valid_0's multi_logloss: 0.296327
[23]	valid_0's multi_logloss: 0.286635
[24]	valid_0's multi_logloss: 0.273357
[25]	valid_0's mul

[210]	valid_0's multi_logloss: 0.10187
[211]	valid_0's multi_logloss: 0.101671
[212]	valid_0's multi_logloss: 0.101593
[213]	valid_0's multi_logloss: 0.101593
[214]	valid_0's multi_logloss: 0.101591
[215]	valid_0's multi_logloss: 0.101482
[216]	valid_0's multi_logloss: 0.101384
[217]	valid_0's multi_logloss: 0.101379
[218]	valid_0's multi_logloss: 0.101379
[219]	valid_0's multi_logloss: 0.101379
[220]	valid_0's multi_logloss: 0.101374
[221]	valid_0's multi_logloss: 0.100926
[222]	valid_0's multi_logloss: 0.100675
[223]	valid_0's multi_logloss: 0.100666
[224]	valid_0's multi_logloss: 0.100661
[225]	valid_0's multi_logloss: 0.100637
[226]	valid_0's multi_logloss: 0.100594
[227]	valid_0's multi_logloss: 0.100514
[228]	valid_0's multi_logloss: 0.100485
[229]	valid_0's multi_logloss: 0.100456
[230]	valid_0's multi_logloss: 0.100439
[231]	valid_0's multi_logloss: 0.100337
[232]	valid_0's multi_logloss: 0.100221
[233]	valid_0's multi_logloss: 0.100221
[234]	valid_0's multi_logloss: 0.10023
[2

[10.4GB(+0.0GB): 10.585sec] fit F


Save the predictions into the same format as required 

In [23]:
all_preds = pd.concat(predictions)
all_preds = all_preds.reindex(subm.index)
all_preds.to_csv('submission.csv')
all_preds.head(20)

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0,92.355790,98.027553
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0,90.518504,101.013442
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0,89.848281,105.576498
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0,90.126965,106.434708
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0,94.274653,110.307909
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000029946,0,80.612268,110.924544
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000040283,0,83.876443,107.727114
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000051343,0,79.438058,106.671816
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000064014,0,84.541710,105.051713
